In [18]:
import cv2
from deepface import DeepFace
import os # Import the os module for path manipulation

# Load your reference image of your dad
reference_img_path = "videos/dad.HEIC" # Ensure this path is correct and the image is accessible

# Load video file (or replace with camera stream)
video_path = "videos/dad_walking.mp4" # Ensure this path is correct and the video is accessible
cap = cv2.VideoCapture(video_path)

frame_count = 0
detected_frames = []

# Create a directory to save detected faces, if it doesn't exist
output_dir = "detected_dad_faces"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Starting facial recognition...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    try:
        # Perform face verification using DeepFace
        # DeepFace.verify returns a dictionary with 'verified' as a key
        # You can also set enforce_detection=False if you want to skip face detection before verification,
        # but it's generally recommended for better accuracy.
        result = DeepFace.verify(img1_path=reference_img_path, img2_path=frame, model_name="VGG-Face", enforce_detection=True)

        if result["verified"]:
            print(f"Dad detected in frame {frame_count}!")
            detected_frames.append(frame_count)

            # Optionally, save the frame where your dad was detected
            output_frame_path = os.path.join(output_dir, f"dad_detected_frame_{frame_count}.jpg")
            cv2.imwrite(output_frame_path, frame)

    except Exception as e:
        # This catch-all handles cases where a face might not be detected in a frame,
        # or other DeepFace-related errors.
        print(f"Could not verify face in frame {frame_count}: {e}")
        continue

cap.release()
cv2.destroyAllWindows() # Close any OpenCV windows
print("\nFinished processing.")
print(f"Dad was detected in a total of {len(detected_frames)} frames.")
print(f"Frames where dad was detected: {detected_frames}")

Starting facial recognition...
Could not verify face in frame 1: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 2: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 3: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 4: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 5: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 6: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 7: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 8: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 9: module 'deepface.modules.modeling' has no attribute 'build_model'
Could not verify face in frame 10: module 'deepface.modules.modeling'

In [ ]:
import cv2
import face_recognition
import numpy as np

# Load known face of your dad
dad_image = face_recognition.load_image_file("firehouse_faces/dad/dad1.jpg")
dad_encoding = face_recognition.face_encodings(dad_image)[0]

# Load video file
video_path = "videos/dad_walking.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
detections = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster processing (optional)
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    rgb_frame = small_frame[:, :, ::-1]  # BGR to RGB

    # Detect face locations and encodings
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for location, encoding in zip(face_locations, face_encodings):
        match = face_recognition.compare_faces([dad_encoding], encoding, tolerance=0.5)[0]
        top, right, bottom, left = location
        if match:
            detections.append({
                "frame": frame_count,
                "location": location
            })
            # Draw box
            cv2.rectangle(small_frame, (left, top), (right, bottom), (0, 255, 0), 2)

    frame_count += 1

    # Display (optional)
    cv2.imshow("Frame", small_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()